In [ ]:
import numpy as np
from itertools import *
import functools as fu
from numpy import kron
from numpy.linalg import cholesky, eig
import torch

from scipy.stats import norm
from qiskit.quantum_info import random_density_matrix
from qutip import rand_dm_hs, Qobj, fidelity

from support_functions import *

## leading dimension to set

In [ ]:
d = 4
local_dim = 2

generating the projectors. Sic and tetra povm

basis and reconstruction basis generation. I'm currenntly focusing on the tetra only

bornvalues

## depolarization and dephasing channel

In [ ]:
def local_depolarizing_channel(qubits_number, density_matrix, probability):

		"""
		NOTE: this function must be used inside a for loop to get to apply each pauli's operator for each state

		"""

		pX = np.array([[0.,1.], [1.,0.]]) # X Pauli matrix
		pY = np.array([[0.,-1.j], [1.j, 0.]]) # Y Pauli matrix
		pZ = np.array([[1., 0.], [0.,-1.]]) # Z Pauli matrix
		pI = np.array([[1.,0.], [0.,1.]])

		for i in range(qubits_number):
			
			operator_string_x = list(repeat(pI,qubits_number))
			operator_string_y = list(repeat(pI,qubits_number))
			operator_string_z = list(repeat(pI,qubits_number))
			

			operator_string_x[i] = pX
			operator_string_y[i] = pY
			operator_string_z[i] = pZ
	
			X = fu.reduce(np.kron,operator_string_x)
			Y = fu.reduce(np.kron,operator_string_y)
			Z = fu.reduce(np.kron,operator_string_z)
		
			new_density_matrix = (1-probability)*density_matrix 
			
			new_density_matrix = new_density_matrix + (probability/3)*X.dot(density_matrix).dot(X)
			#print(new_density_matrix.shape)
			new_density_matrix = new_density_matrix + (probability/3)*Y.dot(density_matrix).dot(Y)

			new_density_matrix = new_density_matrix + (probability/3)*Z.dot(density_matrix).dot(Z)
				
		return new_density_matrix




small test

In [ ]:
def global_depo_channel(dm,p):
    
    dim = dm.shape[0]
    I = np.eye(dim,dim)
    
    return (1-p)*dm + (p/(dim))*I
    

small tests

producing data files. Separable and entangled as usual
Cholesky decomposition included

In [ ]:

def vectorization_new(rho):
	
	dim = rho.shape[0]
	#chol = np.linalg.cholesky(rho)
	chol = 	torch.linalg.cholesky(torch.complex(torch.Tensor(rho.real),torch.Tensor(rho.imag)))

	diag = np.diag(chol).real.tolist()

	lower_t_indeces = np.tril_indices_from(chol,k=-1)

	reals = list(chol[lower_t_indeces].real.flatten())
	imags = list(chol[lower_t_indeces].imag.flatten())

	return np.array(diag+reals+imags, dtype=np.float64)

def vectorization_noChol(rh):

	diag = np.diag(rh).real.tolist()
	lower_t_indeces = np.tril_indices_from(rh,k=-1)

	reals = list(rh[lower_t_indeces].real.flatten())
	imags = list(rh[lower_t_indeces].imag.flatten())

	return np.array(diag+reals+imags, dtype=np.float64)


def from_mpc_tonumpy(colored_rho):

	"""
	input: a density matrix, mpc type
	output: nd.array of the same density matrix recasted in complex128 type
	"""
	placeholder = np.zeros((colored_rho.shape[0],colored_rho.shape[1]),dtype=complex)
	for r in range(colored_rho.shape[0]):
		for c in range(colored_rho.shape[1]):
			a =np.double(mp.nstr(mp.re(colored_rho[r][c])))
			b = np.double(mp.nstr(mp.im(colored_rho[r][c])))
			built = complex(a,b)
			placeholder[r][c] = built
	return placeholder

def eigenvaluesCheck(dm):

	eis, eigvecs = eig(dm)

	eis[eis.real <0 ] = 0.0001
	eis = [ el.real for el in eis]
	eis = np.array(eis)

	cleanRho = eigvecs@ np.diag(eis) @ eigvecs.T.conj()

	return cleanRho/np.trace(cleanRho)
	
def PureEigenvaluesCheck(dm):

	eis, eigvecs = eig(dm)

	eis[eis.real.round(2) < 0.99 ] = 0.0001
	eis = [ el.real for el in eis]
	eis = np.array(eis)

	cleanRho = eigvecs@ np.diag(eis) @ eigvecs.T.conj()

	return cleanRho/np.trace(cleanRho)

## Uploading the n qubits basis

one to generate the frequencies, the dual to reconstruct a density matrix out of them.

In [ ]:


general_basis = np.load("../4qubitsPauli.npy ", allow_pickle=True)
reconstruction_basis = np.load("../4qubitsPauliReconstruction.npy", allow_pickle=True)

In [ ]:
reconstruction_basis.shape

## PAULI basis data generation

In [ ]:
import os
from scipy.stats import norm

totarr =[]
hs = []
tot =1
i = 0
li = []

trials = 1000

normTrials = np.sqrt(trials/(local_dim**(d)))

folder = '../multinomialData/'
filename = 'Haar4qubitsTrials'+str(trials)+'PAULI.npy'

if not os.path.isdir(folder):
    os.mkdir(folder)

def HSdist(A,B):
    return np.trace((A-B)@(A-B)).real
  
hsm = []
while True:

	#PURE STATES
	rho_start0 = np.array(random_density_matrix(local_dim**d,rank=1, method = "Hillbert_Schmidt").data)
	#rho_start0 = global_depo_channel(rho_start0,0.1)
	borns =  np.array([ np.trace(rho_start0 @ base).real for base in general_basis])

	#OLD ONE
	singleFreqVariance = norm.rvs(size = local_dim**(2*d))/18
	#
	borns_approx =  [ el1+el2 for el1,el2 in zip(borns,singleFreqVariance) ]  

	#reconstruct the approximation
	rback = sum([ np.round(reconstruction_basis[i],12)*borns_approx[i] for i in range(4**d) ])
	
	#brute force approximation upon LI
	cleanRho = eigenvaluesCheck(rback)

	chol_exp = [ vectorization_new(cleanRho)  ]

	try:
		#LI test
		li.append(fidelity(Qobj(cleanRho),Qobj(rho_start0)))
		#hs.append(HSdist(cleanRho,rho_start0 ))

		chol_theoretic = vectorization_new(PureEigenvaluesCheck(rho_start0))
		totarr.append( np.concatenate((chol_exp, chol_theoretic), axis=None) )
		i+=1
		#print(i)
	except RuntimeError:
		pass
	
	if i == tot:

		#np.save(folder +filename,np.array(totarr))
		break
	else:
		pass
	

# OUT OF DISTRIBUTION

In [ ]:
import pandas as pd
rm = pd.read_pickle('outofdistribution100.pkl').to_numpy()

rm[0][0].shape[0]

In [ ]:

measurement_error =  norm.rvs(size = local_dim**(2*d))/25
print(np.mean(measurement_error))

In [ ]:
import os
from scipy.stats import norm

totarr =[]
tot = rm.shape[0]
i = 0
li2 = []
#trials = 1000
#normTrials = np.sqrt(trials/(local_dim**(d)))

trials = 100000
#filename = 'sample11-100stepsPauli4qubitsTrials'+str(trials)+'OfD.npy'
filename = 'sample2B-OATPauli85Depo0.3andMeasurementError.npy'


hsm = []
while True:
	
	rmdepo = global_depo_channel(rm[i][0],0.3)
	print(np.trace(rmdepo@rmdepo))
	borns =  np.array([ np.trace(rmdepo @ base).real for base in general_basis])


	#borns_approx = np.random.multinomial(trials, borns)/trials

	#FOR PAULIS plus error measurement
	singleFreqVariance = norm.rvs(size = local_dim**(2*d))/18 + measurement_error
	#FOR PAULIs
	#singleFreqVariance = norm.rvs(size = local_dim**(2*d))/18
	#
	borns_approx =  [ el1+el2 for el1,el2 in zip(borns,singleFreqVariance) ]  

	#reconstruct the approximation
	rback = sum([ np.round(reconstruction_basis[i],12)*borns_approx[i] for i in range(4**d) ])
	#brute force approximation upon LI
	cleanRho = eigenvaluesCheck(rback)

	chol_exp = [ vectorization_new(cleanRho)  ]
	try:
		totarr.append( chol_exp )
		#totarr.append( rback )
		li2.append(fidelity(Qobj(cleanRho ),Qobj(rm[i][0])))
		i+=1
		#print(np.array(totarr).shape),print(i)
	except RuntimeError:
		pass
	
	if i == tot:

		np.save('forPRR/'+filename,np.array(totarr))
		break
	else:
		pass
	


In [ ]:
np.mean(li2), np.std(li2)